<p style="text-align: center; font-size: 40px; font-weight: bold"> Sistemas Distribuídos </p>

<p style="text-align: center; font-size: 30px; font-weight: bold"> Prof. Carlos Bruno </p>

<p style="text-align: center; font-size: 30px; font-weight: bold"> Dupla: Alessandro Souza  e  Hernany Costa </p>

<hr/>

# Classe responsável por fazer a detecção de objetos usando yolo

In [1]:
import cv2
import numpy as np


class ObjectDetection():
    
    def init_yolo(self):
        self.classes = None

        with open('yolov3.txt') as f:
            self.classes = [line.strip() for line in f.readlines()]

        self.COLORS = np.random.uniform(0, 255, size=(len(self.classes), 3))
    
    
    def get_output_layers(self, net): 
        layer_names = net.getLayerNames()
        output_layers = [layer_names[layer[0] - 1] for layer in net.getUnconnectedOutLayers()]
        return output_layers
    
    
    def setup(self, img_name):
        self.init_yolo()
        
        image = cv2.imread(img_name)

        width = image.shape[1]
        height = image.shape[0]
        scale = 0.005

        #carregando configurações da rede neural pré-treinadada
        net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')

        blob = cv2.dnn.blobFromImage(image, 0.005, (416,416), (0,0,0), True, crop=False)

        net.setInput(blob)

        outs = net.forward(self.get_output_layers(net))

        return image,width, height, outs
    
    
    def confidence_calc(self, outs, width, height):
        class_ids = []
        confidences = []
        boxes = []
        conf_threshold = 0.5
        nms_threshold = 0.4

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]

                if confidence > 0.5:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)

                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = center_x - w / 2
                    y = center_y - h / 2

                    class_ids.append(class_id)
                    confidences.append(float(confidence))

                    boxes.append([x, y, w, h])


        indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

        return confidences, indices, class_ids, boxes

    
    def draw_prediction(self, img, class_id, confidence, x, y, x_plus_w, y_plus_h):
        confidence_score = round(confidence, 2)
        label = self.classes[class_id]
        text = '{} {}'.format(label, confidence_score)

        font_scale = .8
        font = cv2.FONT_HERSHEY_PLAIN
        thickness = 1
        color_box = self.COLORS[class_id]
        color_text = (255, 255, 255)

        #desenhando retângulo em volta do objeto
        cv2.rectangle(img, (x, y), (x_plus_w, y_plus_h), color_box, 2)

        #calculando tamanho do texto da label
        (text_width, text_height) = cv2.getTextSize(text, font, font_scale, thickness)[0]

        #outro retângulo pra servir como plano de fundo do texto
        box_coords = ((x, y), (x + text_width, y - text_height))
        cv2.rectangle(img, box_coords[0], box_coords[1], color_box, cv2.FILLED)

        #colocando o texto
        cv2.putText(img, text, (x, y), font, font_scale, color_text, thickness)
        
        
    def draw_prediction_boxes(self, image, indices, confidences, class_ids, boxes):
        predicted_objects = []

        for i in indices:
            i = i[0]
            box = boxes[i]
            x = box[0]
            y = box[1]
            w = box[2]
            h = box[3]
            predicted_objects.append(self.classes[class_ids[i]])

            self.draw_prediction(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))

        cv2.imwrite("object-detection.jpg", image)
        return predicted_objects
    
    def predict(self, img_name):
        image, width, height, outs = self.setup(img_name)
        confidences, indices, class_ids, boxes = self.confidence_calc(outs, width, height)
        predicted_objects = self.draw_prediction_boxes(image, indices, confidences, class_ids, boxes)
        
        return image, predicted_objects
    
    def to_grayscale(self, img_name):
        image = cv2.imread(img_name, 0)
        cv2.imwrite('grayscale.jpg', image)

# Servidor

In [ ]:
import json
import base64
import io

from socket import *
serverPort = 9999
serverSocket = socket(AF_INET, SOCK_STREAM)
serverSocket.bind(('', serverPort))
serverSocket.listen(1)



def get_request(connectionSocket):
    sentence = io.BytesIO()
    buf = connectionSocket.recv(1024)
    sentence.write(buf)
    tamBegin = -1
    tamEnd = -1

    #verificação para encontrar o tamanho total dos dados que serão enviados
    print('Recebendo dados')
    i = 0
    while (i<= len(buf)):
        if(tamBegin != (-1) and tamEnd != (-1) ):
            break;
        if(sentence.getvalue()[2:].decode()[i] == ":" and tamBegin == -1):
            tamBegin = i+1
        elif( sentence.getvalue()[2:].decode()[i] == ":" and tamEnd == -1):
            tamEnd = i
        i=i+1
    
    data_size = int(sentence.getvalue()[2:].decode()[tamBegin:tamEnd])+tamEnd+3
    print('DataSize =', data_size)
    
    #lendo até que tenham sido lidos todos os dados
    data_size -= len(buf)
    while (data_size > 0):
        buf = connectionSocket.recv(data_size)
        data_size -= len(buf)
        sentence.write(buf)
    
    x = json.loads(sentence.getvalue()[tamEnd+3:].decode())
    print('Dados recebidos com sucesso!')
    
    return x


def send_reply(request, socket):
    detector = ObjectDetection()
    
    request_id = request['requestID']
    object_reference = request['objectReference']
    method_id = request['methodID']
    message_type = request['messageType']
    
    if message_type != 0 or object_reference != 'ObjectDetection'  or method_id not in [1, 2]:
        print('Mensagem com erro!\nRequestId: {}\nMethodId: {}\nobjectReference: {}'.format(request_id, method_id, object_reference))
        return
    
    
    image = request['arguments']
    img_name = 'imageToSave.jpg'
    
    #salvando imagem no disco
    fh = open(img_name, "wb")
    fh.write(base64.decodebytes(image.encode('utf-8')))
    fh.close()

    #identificar objetos
    if method_id == 1:
        
        print('Iniciando predição')
        image, predicted_objects = detector.predict(img_name)

        print('RequestID', request_id, '- Foram encontrados:' ,  predicted_objects)
        
        with open("object-detection.jpg", "rb") as imageFile:
            img_str = base64.b64encode(imageFile.read())

        request['arguments'] = img_str.decode()
        
    
    #aplicar filtro preto e branco
    if method_id == 2:
        
        
        detector.to_grayscale(img_name)

        print('RequestID', request_id, '- Filtro aplicado')
        
        with open("grayscale.jpg", "rb") as imageFile:
            img_str = base64.b64encode(imageFile.read())

        request['arguments'] = img_str.decode()
        
    
    request['messageType'] = 1
    
    #simulando caso em que o requestID não é o mesmo da requisição
    #pego um número no range de [0,2[ (que contém só 0 e 1)
    #sendo que o 0 tem 80% de chance de ser escolhido e o 1 tem 20%
    num = np.random.choice(2, 1, p=[.8, .2])
    
    #caso tenha escolhido o 1, atero o requestID para um valor inválido
    if(num == 1):
        print('requestID alterado')
        request['requestID'] = -1
    
    print('\n')
    socket.send((json.dumps(request)).encode())
        
        
print("Esperando")
while True:
    connectionSocket, addr = serverSocket.accept()   
    
    request = get_request(connectionSocket)
    
    send_reply(request, connectionSocket)
    
    connectionSocket.close()

Esperando
